# PolyLogToolkit

*LogToolkit* is a script implemented to support symbolic computations of multiple polylogarithms, detailed in Haoran Li's dissertation [Hopf Algebra of Multiple Polylogarithms and Its Associated One Forms](https://lihaoranicefire.github.io/math/LogToolKit/HopfAlgebraOfMultiplePolylogarithmsAndItsAssociatedOneForms.pdf)

## Implementation

In [2]:
import itertools
from sympy import *

# Reserve some symbol heads
II = IndexedBase('II')
Li = IndexedBase('Li')
du = IndexedBase('du')
dv = IndexedBase('dv')

#### $\mathbb I^{\text{Symb}}$

$$
I(a_{i_0}; 0^{n_0-1}, a_{i_1}, 0^{n_1-1}, \cdots, a_{i_m}, 0^{n_m-1}; a_{i_{m+1}})\leftrightsquigarrow (i_0, n_0, i_1, n_1, \cdots, i_m, n_m, i_{m+1})
$$

for example, in depth $2$, $I(a_0; 0^{1-1}, a_1, 0^{3-1}, a_2, 0^{2-1}; a_3)\leftrightsquigarrow(0,1,1,3,2,2,2)$

In [32]:
class ISymb(Indexed):
    def __new__(cls, *args):
        return super().__new__(cls, II, *args)
    def __init__(self, *args):
        # self.args start with II
        self.m = len(self.args) // 2 - 2
        self.i = self.args[1::2]
        self.n = self.args[2::2]
    def __lt__(self, other):
        pass
    def __repr__(self):
        return f'II{self.args[1:]}'

#### $\mathbb H^{\text{Symb}}$

$$
[x_{i_1\to i_2},\cdots,x_{i_d\to i_{d+1}}]_{n_1,\cdots,n_d}\leftrightsquigarrow(i_1,n_1,i_2-i_1,\cdots,i_d-i_{d-1},n_d,i_{d+1}-i_d)
$$

Or $(m_1,n_1,\cdots,m_d,n_d,m_{d+1})$ so that $i_r = m_1 + \cdots + m_r$

The $\partial_{r}$ of $(m_1,n_1,\cdots,m_d,n_d,m_{d+1})$ is
- if $d == 1$ and $n_1 == 1$: $-dv_{i_1,i_2-1}$
- else if $n_r>1$ or $d=1$: $(\cdots,n_r-1,\cdots) du_{i_r, i_{r+1}-1}$
- else if $r=d$: $-(\cdots,n_{d-1},m_d+1+m_{d+1}) dv_{i_d,i_{d+1}-1}$
- else: $-(\cdots,m_{r}+1+m_{r+1},n_{r+1},\cdots) dv_{i_r, i_{r+1}-1} + (\cdots,m_r,n_{r+1},m_{r+1}+1+m_{r+2},\cdots) (du_{i_r, i_{r+1}-1} - dv_{i_r, i_{r+1}-1})$

In [33]:
class HSymb(Indexed):
    def __new__(cls, *args):
        return super().__new__(cls, Li, *args)
    def __init__(self, *args):
        # self.args start with Li
        self.d = len(args) // 2
        self.m = (0, *args[::2])
        self.n = (0, *args[1::2])
        self.i = list(itertools.accumulate(self.m))
    def __lt__(self, other):
        pass
    def __repr__(self):
        return f'Li{self.args[1:]}'
    def partial_differential(self, r):
        '''
        Take the partial_r of a polylogarithm
        '''
        if self.d == 1 and self.n[1] == 1:
            return -dv[self.i[1], self.i[2] - 1]
        if self.n[r] > 1 or self.d == 1:
            return HSymb(*self.args[1:2*r-1], self.n[r] - 1, *self.args[2*r+1:]) * du[self.i[r], self.i[r+1] - 1]
        elif r == self.d:
            return -HSymb(*self.args[1:-3], self.m[r] + 1 + self.m[r+1]) * dv[self.i[r], self.i[r+1] - 1]
        else:
            return -HSymb(*self.args[1:2*(r-1)], self.m[r] + 1 + self.m[r+1], *self.args[2*(r+1):]) * dv[self.i[r], self.i[r+1] - 1] +\
                    HSymb(*self.args[1:2*r-1], self.n[r+1], self.m[r+1] + 1 + self.m[r+2], *self.args[2*(r+2):]) *\
                    (du[self.i[r], self.i[r+1] - 1] - dv[self.i[r], self.i[r+1] - 1])
    def differential(self):
        '''
        Take the differential of a polylogarithm
        '''
        return sum(self.partial_differential(r) for r in range(1, self.d + 1))

#### Differential

encodings rendered by successive partial derivatives. The $\partial_{r}$-terms of $(m_1,n_1,\cdots,m_d,n_d,m_{d+1})$ include
- if $n_r>1$ or $d=1$: $(\cdots,n_r-1,\cdots)$
- else if $r=d$: $(\cdots,n_{d-1},m_d+1+m_{d+1})$
- else: $(\cdots,m_{r}+1+m_{r+1},n_{r+1},\cdots)$ and $(\cdots,m_r,n_{r+1},m_{r+1}+1+m_{r+2},\cdots)$

In [5]:
def differential(expr):
    '''
    Take the differential of an element in HSymb
    '''
    if expr.is_Add:
        return sum(differential(arg) for arg in expr.args)
    elif expr.is_Mul:
        return sum(expr / arg * differential(arg) for arg in expr.args if not arg.is_number)
    elif expr.is_Pow:
        return expr.args[1] * expr.args[0] ** (expr.args[1] - 1) * differential(expr.args[0])
    elif expr.is_number:
        return 0
    elif isinstance(expr, HSymb):
        return expr.differential()

#### Tensor

$$
\left(\sum_{i_1}c_{i_i}^{(1)}t_{i_1}^{(1)}\right)\otimes\cdots\otimes\left(\sum_{i_N}c_{i_N}^{(N)}t_{i_N}^{(N)}\right)=\sum_{i_1,\cdots,i_N}c_{i_i}^{(1)}\cdots c_{i_N}^{(N)} t_{i_1}^{(1)}\otimes\cdots\otimes t_{i_N}^{(N)}
$$

$$
\left(\sum_{\mathbf i}c_{\mathbf i}^{(1)}t_{i_1}^{(1)}\otimes\cdots\otimes t_{i_N}^{(1)}\right)\left(\sum_{\mathbf j}c_{\mathbf j}^{(2)}t_{j_1}^{(2)}\otimes\cdots\otimes t_{j_N}^{(2)}\right)=\sum_{i_1,\cdots,i_N,j_1,\cdots,j_N}c_{\mathbf i}^{(1)}c_{\mathbf j}^{(2)}\left(t_{i_1}^{(1)}t_{j_1}^{(2)}\right)\otimes\cdots\otimes\left(t_{i_N}^{(1)}t_{j_N}^{(2)}\right)
$$

In [100]:
class tensor():
    def __init__(self, *args):
        self.is_number = False
        # Giving coefficients and terms already
        if len(args) == 2 and all(isinstance(arg, (list, tuple)) for arg in args):
            self.c, self.t = args
        # Giving the arguments of a single raw tensor
        else:
            # expand the args first
            args = [expand(arg) for arg in args]

            self.c, self.t = [], []
            for comp in list(itertools.product(*map(tensor._split_coef, args))):
                c, t = zip(*comp)
                self.c.append(prod(c))
                self.t.append(t)

    @staticmethod
    def _split_coef(expr):
        if expr.is_Add:
            return [tensor._split_coef(arg)[0] for arg in expr.args]
        elif expr.is_number:
            return [[expr, 1]]
        elif expr.is_Mul:
            coef = prod(arg for arg in expr.args if arg.is_number)
            return [[coef, expr / coef]]
        else:
            return [[1, expr]]

    def __add__(self, other):
        return tensor(self.c + other.c, self.t + other.t)
    def __neg__(self):
        return tensor(list(map(lambda x: -x, self.c)), self.t)
    def __sub__(self, other):
        return self + (-other)

    @staticmethod
    def _mul(t1, t2):
        if not isinstance(t1, (list, tuple)) or not isinstance(t2, (list, tuple)):
            raise ValueError('Must be a list or a tuple')
        if len(t1) != len(t2):
            raise ValueError('Dimension Mismatch')
        return tuple(map(prod, zip(t1, t2)))
    def __rmul__(self, other):
        # Scalar multiplication (scalar on the left side)
        if isinstance(other, (int, float)) or other.is_number:
            return tensor([c * other for c in self.c], self.t)
        return self * other
    def __mul__(self, other):
        if isinstance(other, (int, float)) or other.is_number:
            return tensor([c * other for c in self.c], self.t)
        return tensor(list(map(prod, itertools.product(self.c, other.c))),
                      [tensor._mul(t1, t2) for t1, t2 in itertools.product(self.t, other.t)])
    def __repr__(self):
        ct = []
        for c, t in zip(self.c, self.t):
            if c == 1:
                ct.append(f"{t}")
            elif c == -1:
                ct.append(f"-{t}")
            else:
                ct.append(f"{c}*{t}")

        return ' + '.join(ct)

In [101]:
tensor(ISymb(1), ISymb(2,3)) * tensor(2 * ISymb(4), ISymb(1))

2*(II[1]*II[4], II[1]*II[2, 3])

In [96]:
tensor(2 * ISymb(4), 2 * ISymb(1), I * ISymb(3))

4*I*(II(4,), II(1,), II(3,))

In [95]:
tensor(2 * ISymb(4), 2 * ISymb(1)) + tensor(I * ISymb(3))

4*(II(4,), II(1,)) + I*(II(3,),)

In [93]:
-2* tensor(2 * ISymb(4), 2 * ISymb(1))

-8*(II(4,), II(1,))

In [94]:
tensor(ISymb(1), ISymb(2,3)) + tensor(2 * ISymb(4), ISymb(1))

(II(1,), II(2, 3)) + 2*(II(4,), II(1,))

#### Wedge

In [ ]:
class wedge:
    def __init__(self, *comp):
        self.comp = comp
    def __repr__(self):
        return f'{self.comp}'
    def expand(self):
        pass

In [81]:
type((ISymb(1,2,3).args)[0])

sympy.tensor.indexed.IndexedBase

In [46]:
simplify(ISymb(1) * ISymb(2) + 2 * ISymb(2,3,4) ** 2).args

(2*II[2, 3, 4]**2, II[1]*II[2])

In [64]:
type(simplify(ISymb(1) * (2*I)).args[-1])

sympy.tensor.indexed.Indexed

In [124]:
HSymb(1,1,1)

TypeError: HSymb.__init__() takes 1 positional argument but 4 were given

In [191]:
differential(2 * I * HSymb(1,1,1))

SympifyError: SympifyError: "cannot sympify object of type <class 'generator'>"

In [68]:
(Rational(2, 3) * ISymb(1) * Rational(1, 5)).is_Mul

True

In [44]:
tensor(ISymb(1), ISymb(2,3)) * tensor(ISymb(4), ISymb(1))

$[II[1]*II[4], II[1]*II[2, 3]]$

In [42]:
(II[1] * II[2]).doit()

II[1]*II[2]

In [ ]:
ISymb(IGen(1)) * ISymb(IGen(2)) + ISymb(IGen(2,3,4)) ** 2

ISymb(I[1]*I[2] + I[2, 3, 4]**2, I[1], I[2], I[2, 3, 4], domain='QQ')

In [ ]:
(Poly(I[0]*I[1])).degree()

1

In [ ]:
Expr

type

In [ ]:
isinstance(I[1]*I[0]+I[2]**2, Expr)

True